In [ ]:
!pip install ultralytics
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor, ToPILImage

In [ ]:
from torchvision.transforms import ToPILImage

def fix_channels(t):
    """
    Ensures tensor image has 3 channels (RGB).
    Handles grayscale, grayscale with channel, and RGBA formats.
    Returns a PIL image.
    """
    if len(t.shape) == 2:
        # (H, W) -> (3, H, W)
        return ToPILImage()(t.expand(3, -1, -1))
    if t.shape[0] == 4:
        return ToPILImage()(t[:3])
    if t.shape[0] == 1:
        return ToPILImage()(t.expand(3, -1, -1))
    return ToPILImage()(t)

In [ ]:
import random
def xyxy_to_xcycwh(box):
    """
    Boxes in images may have the format (x1, y1, x2, y2) and we may need the format (center of x, center of y, width, height).
    :param box: Tensor-like box with format (x1, y1, x2, y2)
    :return: Tensor-like box with format (center of x, center of y, width, height)
    """
    x1, y1, x2, y2 = box.unbind(dim=1)
    width = x2-x1
    height = y2-y1
    xc = x1 + width*0.5
    yc = y1 + height*0.5
    b = [xc, yc, width, height]
    return torch.stack(b, dim=1)

def cxcywh_to_xyxy(x):
    """
    Boxes in images may have the format (center of x, center of y, width, height) and we may need the format (x1, y1, x2, y2).
    :param box: Tensor-like box with format (center of x, center of y, width, height)
    :return: Tensor-like box with format (x1, y1, x2, y2)
    """
    x_c, y_c, w, h = x.unbind(1)
    x1 = x_c - 0.5 * w
    y1 = y_c - 0.5 * h
    x2 = x_c + 0.5 * w
    y2 = y_c + 0.5 * h
    b = [x1, y1, x2, y2]
    return torch.stack(b, dim=1)

In [ ]:
from datasets import load_dataset, ReadInstruction

train_dataset = load_dataset(
    "detection-datasets/fashionpedia",
    split=ReadInstruction("train", from_=0, to=95, unit="%", rounding="pct1_dropremainder")
)
val_dataset = load_dataset(
    "detection-datasets/fashionpedia",
    split=ReadInstruction("train", from_=95, to=100, unit="%", rounding="pct1_dropremainder")
)

README.md:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

(…)-00000-of-00007-fe108070118553c3.parquet:   0%|          | 0.00/482M [00:00<?, ?B/s]

(…)-00001-of-00007-f41a5a9c38c9005b.parquet:   0%|          | 0.00/480M [00:00<?, ?B/s]

(…)-00002-of-00007-40bc8456894bcbcd.parquet:   0%|          | 0.00/480M [00:00<?, ?B/s]

...

(…)-00005-of-00007-41d8dfe1edb6591e.parquet:   0%|          | 0.00/487M [00:00<?, ?B/s]

(…)-00006-of-00007-f41b0f2f4bbefac9.parquet:   0%|          | 0.00/487M [00:00<?, ?B/s]

(…)-00000-of-00001-0b29e85429788213.parquet:   0%|          | 0.00/84.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45623 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1158 [00:00<?, ? examples/s]

In [ ]:
cats = train_dataset.features['objects'].feature['category']
cats

def idx_to_text(indexes):
    """
    Converts an index into a category label.
    :param indexes: List of indexes
    :return: List of category labels
    """
    labels = []
    for i in indexes:
        labels.append(cats.names[i])
    return labels

In [ ]:
from PIL import Image, UnidentifiedImageError

def safe_save_image(image, path, size=(640, 640)):
    try:
        img = image.convert('RGB')
        img = img.resize(size, Image.BILINEAR)
        img.save(path, 'JPEG')
        return True
    except (UnidentifiedImageError, OSError) as e:
        print(f"Corrupt image skipped: {path} ({e})")
        return False

In [ ]:
def normalize_and_validate_bbox(bbox, width, height):
    x, y, w, h = bbox
    x_center = (x + w / 2) / width
    y_center = (y + h / 2) / height
    w_norm = w / width
    h_norm = h / height

    # Clip to [0, 1]
    x_center = min(max(x_center, 0), 1)
    y_center = min(max(y_center, 0), 1)
    w_norm = min(max(w_norm, 0), 1)
    h_norm = min(max(h_norm, 0), 1)

    # Only accept boxes with positive area and within bounds
    if 0 < w_norm <= 1 and 0 < h_norm <= 1:
        return x_center, y_center, w_norm, h_norm
    else:
        return None

In [ ]:
from tqdm import tqdm
import os

def save_yolo_format(dataset, split='train', base_path='datasets/fashion_yolo', img_size=(640, 640)):
    img_dir = os.path.join(base_path, 'images', split)
    lbl_dir = os.path.join(base_path, 'labels', split)
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(lbl_dir, exist_ok=True)

    for i, data in enumerate(tqdm(dataset, desc=f"Saving {split} set")):
        image = data['image']
        width, height = image.size
        image_path = os.path.join(img_dir, f'{i}.jpg')
        label_path = os.path.join(lbl_dir, f'{i}.txt')

        # Validate and save image
        if not safe_save_image(image, image_path, size=img_size):
            continue  # Skip this image if it can't be saved

        bboxes = data['objects']['bbox']
        categories = data['objects']['category']
        valid_lines = []
        for bbox, category in zip(bboxes, categories):
            norm = normalize_and_validate_bbox(bbox, width, height)
            if norm:
                x_center, y_center, w_norm, h_norm = norm
                valid_lines.append(f"{category} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

        # Write label file (empty if no valid boxes)
        with open(label_path, 'w') as f:
            f.write('\n'.join(valid_lines))

In [ ]:
save_yolo_format(train_dataset, split='train')
save_yolo_format(val_dataset, split='val')

Saving val set: 100%|██████████| 2280/2280 [00:39<00:00, 57.90it/s]


In [ ]:
import yaml

FASHION_CLASSES = {
    0: "shirt, blouse", 1: "top, t-shirt, sweatshirt", 2: "sweater", 3: "cardigan",
    4: "jacket", 5: "vest", 6: "pants", 7: "shorts", 8: "skirt", 9: "coat", 10: "dress",
    11: "jumpsuit", 12: "cape", 13: "glasses", 14: "hat", 15: "headband, head covering, hair accessory",
    16: "tie", 17: "glove", 18: "watch", 19: "belt", 20: "leg warmer", 21: "tights, stockings",
    22: "sock", 23: "shoe", 24: "bag, wallet", 25: "scarf", 26: "umbrella", 27: "hood",
    28: "collar", 29: "lapel", 30: "epaulette", 31: "sleeve", 32: "pocket", 33: "neckline",
    34: "buckle", 35: "zipper", 36: "applique", 37: "bead", 38: "bow", 39: "flower",
    40: "fringe", 41: "ribbon", 42: "rivet", 43: "ruffle", 44: "sequin", 45: "tassel"
}

# Prepare list of class names in order (0 to 45)
class_names = [FASHION_CLASSES[i] for i in range(len(FASHION_CLASSES))]

import os

base_path = '/content/datasets/fashion_yolo'
data_yaml = {
    'train': os.path.join(base_path, 'images/train'),
    'val': os.path.join(base_path, 'images/val'),
    'nc': len(class_names),
    'names': class_names
}

import yaml
with open(os.path.join(base_path, 'data.yaml'), 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

In [ ]:
from ultralytics import YOLO

# Load the base model (YOLOv8 nano here, change if needed)
model = YOLO('yolov8n.pt')

# Train for 15 epochs
model.train(
    data='/content/datasets/fashion_yolo/data.yaml',
    epochs=15,
    imgsz=640,
    batch=16,
    project='fashion_yolo_project',
    name='yolov8n_fashion',
    resume=False  # Make sure we start fresh
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 124MB/s]


Ultralytics 8.3.151 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/fashion_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_fashion, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

100%|██████████| 755k/755k [00:00<00:00, 19.1MB/s]

Overriding model.yaml nc=80 with nc=46

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

      15/15      3.91G     0.8419      1.457      1.511         67        640: 100%|██████████| 2708/2708 [13:09<00:00,  3.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:23<00:00,  3.07it/s]


                   all       2280      16077      0.463      0.381      0.319       0.25

15 epochs completed in 3.501 hours.
Optimizer stripped from fashion_yolo_project/yolov8n_fashion/weights/last.pt, 6.3MB
Optimizer stripped from fashion_yolo_project/yolov8n_fashion/weights/best.pt, 6.3MB

Validating fashion_yolo_project/yolov8n_fashion/weights/best.pt...
Ultralytics 8.3.151 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,014,618 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:29<00:00,  2.48it/s]


                   all       2280      16077      0.462      0.381      0.319       0.25
         shirt, blouse        395        397      0.469      0.702      0.647       0.41
top, t-shirt, sweatshirt        742        759      0.448      0.754      0.632      0.501
               sweater         96         98      0.362      0.378      0.313      0.269
              cardigan         64         64      0.292      0.219      0.219       0.19
                jacket        416        420      0.502       0.85      0.793      0.692
                  vest         34         34          0          0     0.0674     0.0378
                 pants        652        652      0.466      0.929      0.766      0.688
                shorts         81         81      0.381      0.778      0.491       0.41
                 skirt        179        179      0.324      0.676      0.434      0.375
                  coat        143        144       0.37       0.66      0.465      0.395
                 dr

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79eb346db8d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,

In [ ]:
from google.colab import files
files.download('/content/fashion_yolo_project/yolov8n_fashion/weights/best.pt')
files.download('/content/fashion_yolo_project/yolov8n_fashion/weights/last.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>